# Processamento de Consulta

limpando o input da consulta

In [27]:
pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=f2a309e8d09339f277cdcba26ae4f7ced33d1f3ddb512deedd77915e461c39dc
  Stored in directory: /home/joao/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
import unidecode
from nltk.corpus import stopwords

def clear_query_input(query, field=None, usestopwords=False):

    # separa as palavras da query
    if type(query) == str:
        substr = re.findall(r'"(.*?)"', query)
        words = re.sub(r'"(.*?)"', '', query)
        words = words.lower().strip().split(' ')
        words = words + substr
   
    # remove simbolos da query
    toRemove = r'[.*,;\(\)\'\"\?\!%\$]'
    for i in range(len(words)):
        words[i] = re.sub(toRemove, '', words[i])
        words[i] = unidecode.unidecode(words[i])
    
    # coloca os stop words em caso decidirmos usa-los 
    if usestopwords:
        set_sw = set(stopwords.words('english'))
        newWords = []
        for word in words:
            if not word in set_sw:
                if (field):
                    newWords.append(word + '.' + field)
                else:
                    newWords.append(word)
        return list(set(newWords))
    
    # caso ele seja a consulta seja de um campo especifico, ele é atribuido as palavras
    if (field):
        for i in range(len(words)):
            words[i] = words[i] + '.' + field
    return list(set(words))

In [2]:
exemplo = 'Uma pequena consulta'
print(clear_query_input(exemplo))
print(clear_query_input(exemplo, field='title'))

['consulta', 'uma', 'pequena']
['uma.title', 'pequena.title', 'consulta.title']


# Ranqueamento

usando os pesos com tfidf: tf * idf
    para o termo-documento = f(i,j) * log(N/ni)
    para a termo-consulta = (0.5 * 0.5(f(i,q)/max(i)f(i,q)) * log(N/ni)
   

In [3]:
from numpy import log

def get_idf(posting, num_docs):

    idf = log(num_docs/len(posting))
    return idf

In [4]:
def cosine_Score(terms, tfidf, invlist, num_docs, doc_lengths, n):
    scores = {i: 0 for i in range(1, num_docs + 1)}
    
    #term-at-a-time
    for term in terms:
        w_term_query = term.count(term)
        if (tfidf):
            w_term_query = 0.5 + 0.5 * \
                (w_term_query /
                 max([i for i, val in enumerate(terms) if val == term])+1)
            w_term_query *= get_idf(invlist[term], num_docs)
        for post in term:
            w_doc_term = post[1]  # tf
            if tfidf:
                w_doc_term *= get_idf(invlist[term], num_docs)
            scores[post[0]] += w_term_query * w_doc_term
    for d in range(1, num_docs + 1):
        scores[d] /= doc_lengths[d]
    scores = sorted(scores.items, key=lambda x: x[1], reverse=True)
    return scores[:n]

In [7]:
import json
from bs4 import BeautifulSoup

def get_docs_len(docs_path_map='./data/data_map.json'):
    lengths = {}
    with open(docs_path_map, 'r') as map:
        map_docs = map.read()
        map_docs = json.loads(map_docs)
    for id in map_docs:
        with open(f"../data/htmls/{map_docs[id]['arquivo']}.html") as html:
            lengths[id] = len(BeautifulSoup(
                html.read(), 'html.parser').text.split(' '))
    return lengths

In [12]:
# pegando tamanho dos arquivos
docs_len = get_docs_len(docs_path_map='../data/data_map.json')

14895


In [15]:
#


UsageError: Line magic function `%cmd` not found.
